In [38]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta
from pandas.tseries.offsets import *
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

year = 2025
quarter = 1
today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-05-18 2025-05-17


In [44]:
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(today, yesterday)

2025-05-18 2025-05-16


### Today must be last business day

In [47]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
yesterday = yesterday.date()
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2025-05-18
Today in mm-dd format: 05-18
yesterday: 2025-05-16


In [49]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

In [51]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [53]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}")

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Ad hoc
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### New dividend stock

In [4]:
name = 'WHAIR'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [3]:
sqlIns = """
INSERT INTO dividend
VALUES('xxx',.1738,0.1738,0.1738,0.1556,0,0,0,00,'2022-05-25','2022-06-22','',0)
"""
rp = const.execute(sqlIns)
rp.rowcount

ObjectNotExecutableError: Not an executable object: "\nINSERT INTO dividend\nVALUES('xxx',.1738,0.1738,0.1738,0.1556,0,0,0,00,'2022-05-25','2022-06-22','',0)\n"

### Start of Update dividend

In [14]:
def update_dividend1(q1, XDATE, PAIDDATE, actual, name):
    # Use parameterized query to avoid SQL injection
    sql = text("""
        UPDATE dividend 
        SET q1 = :q1, 
            dividend = q1 + q2 + q3 + q4, 
            XDATE = :XDATE, 
            PAIDDATE = :PAIDDATE, 
            actual = :actual 
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sql, {
        'q1': q1,
        'XDATE': XDATE,
        'PAIDDATE': PAIDDATE,
        'actual': actual,
        'name': name
    })
    const.commit()
    return f"Records updated = {rp.rowcount}"

In [28]:
name = 'GVREIT'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,GVREIT,0.2044,0.1911,0.1963,0.2050,0.7968,"75,000",2025-02-26,2025-03-12,,1


In [34]:
q1 = 0.2050
XDATE = '2025-05-28'
PAIDDATE = '2025-06-12'
actual = 1

update_dividend1(q1,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [36]:
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,GVREIT,0.2044,0.1911,0.1963,0.2050,0.7968,"75,000",2025-05-28,2025-06-12,,1


In [27]:
def update_dividend2(shares, q1, actual, name):
    # Use parameterized query to avoid SQL injection
    sql = text("""
        UPDATE dividend 
        SET q1 = :q1, 
            dividend = q1 + q2 + q3 + q4, 
            shares = :shares, 
            actual = :actual 
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sql, {
        'q1': q1,
        'shares': shares,
        'actual': actual,
        'name': name
    })
    
    return f"Records updated = {rp.rowcount}"

In [31]:
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,GVREIT,0.2044,0.1911,0.1963,0.2050,0.7968,"60,000",2025-02-26,2025-03-12,,1


In [9]:
shares = 20000
q1 = 0
actual = 0

In [11]:
update_dividend2(shares,q1,actual,name)

'Records updated = 1'

### Toggle actual status

In [2]:
name = 'BCT'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete dividend record

In [10]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("MC")
'''
#rp = const.execute(sqlDel)
rp.rowcount

1

### Start of output to cloud

In [14]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct actual'.split()

In [16]:
sql = """
SELECT Y.NAME AS name,  Q1 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q1 > 0 
AND P.date = '%s'
ORDER BY name
"""
sql = sql % yesterday 
print(sql)


SELECT Y.NAME AS name,  Q1 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q1 > 0 
AND P.date = '2025-05-16'
ORDER BY name



In [18]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,actual
0,AIMIRT,0.2150,"12,500","2,687.50","2,418.75",2025-05-19,2025-06-06,"136,250.00",1.78%,1
1,CPNREIT,0.2505,"55,000","13,777.50","12,399.75",2025-05-19,2025-06-04,"990,000.00",1.25%,1
2,DIF,0.2222,"40,000","8,888.00","7,999.20",2025-05-14,2025-06-09,"524,000.00",1.53%,1
3,GVREIT,0.2050,"75,000","15,375.00","13,837.50",2025-05-28,2025-06-12,"573,000.00",2.41%,1
4,IVL,0.1750,"7,200","1,260.00","1,134.00",2025-05-27,2025-06-12,"288,000.00",0.39%,1
5,TFFIF,0.1189,"20,000","2,378.00","2,140.20",2025-05-28,2025-06-16,"150,000.00",1.43%,1
6,WHAIR,0.1325,"60,000","7,950.00","7,155.00",2025-05-26,2025-06-25,"481,998.00",1.48%,1
7,WHART,0.1915,"20,000","3,830.00","3,447.00",2025-05-19,2025-06-06,"246,000.00",1.40%,1


In [20]:
dividend.net.sum()

50531.399999999994

In [26]:
file_name = 'dividend-q1.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\dividend-q1.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\dividend-q1.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\dividend-q1.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\dividend-q1.csv


In [30]:
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(god_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(osd_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(icd_file, index=False)

In [10]:
file_name = 'dividend-q1.xlsx'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name
one_file = one_path + file_name

dividend[cols].sort_values(['name'],ascending=[True]).to_excel(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(data_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(osd_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(one_file, index=False)

### End of output to cloud

In [67]:
sql = 'SELECT name, q_eps, aq_eps, publish_date FROM epss WHERE year = %s-1 AND quarter = %s ORDER BY name'
sql = sql % (year, quarter)
print(sql)
eps = pd.read_sql(sql, conlt)
eps.dtypes

SELECT name, q_eps, aq_eps, publish_date FROM epss WHERE year = 2025-1 AND quarter = 1 ORDER BY name


name             object
q_eps           float64
aq_eps          float64
publish_date     object
dtype: object

In [69]:
df_merge = pd.merge(dividend, eps, on='name', how='inner')
df_merge.sort_values('name',ascending=True)

,name,qtrly,shares,xdate,paiddate,price,yearly,actual,cost_amt,amount,amt_yearly,net,pct,q_eps,aq_eps,publish_date
0,AIMIRT,0.2150,12500,2025-05-19,2025-06-06,9.70,7.23,1,136250.0,2687.5,9853.75,2418.75,1.78,0.00,0.00,2024-05-15
1,CPNREIT,0.0987,55000,2025-05-19,2025-06-04,12.30,3.17,1,990000.0,5428.5,31377.50,4885.65,0.49,0.00,0.00,2024-05-15
2,DIF,0.2222,40000,2025-05-14,2025-06-09,8.00,6.78,1,524000.0,8888.0,35552.00,7999.20,1.53,0.00,0.00,2024-05-14
3,GVREIT,0.2044,75000,2025-05-28,2025-06-12,6.85,10.46,1,573000.0,15330.0,59917.50,13797.00,2.41,0.00,0.00,2024-02-15
4,IVL,0.1750,7200,2025-05-27,2025-06-12,19.70,1.75,1,288000.0,1260.0,5040.00,1134.00,0.39,0.17,0.17,2024-05-10
5,TFFIF,0.1039,20000,2025-05-28,2025-06-16,6.05,6.13,1,150000.0,2078.0,9192.00,1870.20,1.25,0.00,0.00,2024-02-13
6,WHAIR,0.0582,60000,2025-05-26,2025-06-25,4.70,6.86,1,481998.0,3492.0,33048.00,3142.80,0.65,0.00,0.00,2024-05-13
7,WHART,0.1358,20000,2025-05-19,2025-06-06,9.50,5.80,1,246000.0,2716.0,14266.00,2444.40,0.99,0.00,0.00,2024-05-09


In [59]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['amt_yearly'] = round(dividend['shares'] * dividend['yearly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend['yearly'] = round(dividend['amt_yearly'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,amt_yearly,yearly,actual
0,AIMIRT,0.2150,"12,500","2,687.50","2,418.75",2025-05-19,2025-06-06,"136,250.00",1.78%,9853.750000,7.230000,1
1,CPNREIT,0.0987,"55,000","5,428.50","4,885.65",2025-05-19,2025-06-04,"990,000.00",0.49%,31377.500000,3.170000,1
2,DIF,0.2222,"40,000","8,888.00","7,999.20",2025-05-14,2025-06-09,"524,000.00",1.53%,35552.000000,6.780000,1
3,GVREIT,0.2044,"75,000","15,330.00","13,797.00",2025-05-28,2025-06-12,"573,000.00",2.41%,59917.500000,10.460000,1
4,IVL,0.1750,"7,200","1,260.00","1,134.00",2025-05-27,2025-06-12,"288,000.00",0.39%,5040.000000,1.750000,1
5,TFFIF,0.1039,"20,000","2,078.00","1,870.20",2025-05-28,2025-06-16,"150,000.00",1.25%,9192.000000,6.130000,1
6,WHAIR,0.0582,"60,000","3,492.00","3,142.80",2025-05-26,2025-06-25,"481,998.00",0.65%,33048.000000,6.860000,1
7,WHART,0.1358,"20,000","2,716.00","2,444.40",2025-05-19,2025-06-06,"246,000.00",0.99%,14266.000000,5.800000,1


In [71]:
colt = 'name qtrly shares amount net publish_date xdate paiddate q_eps aq_eps'.split()

In [73]:
df_merge[colt].sort_values('publish_date',ascending=True)

,name,qtrly,shares,amount,net,publish_date,xdate,paiddate,q_eps,aq_eps
5,TFFIF,0.1039,20000,2078.0,1870.20,2024-02-13,2025-05-28,2025-06-16,0.00,0.00
3,GVREIT,0.2044,75000,15330.0,13797.00,2024-02-15,2025-05-28,2025-06-12,0.00,0.00
7,WHART,0.1358,20000,2716.0,2444.40,2024-05-09,2025-05-19,2025-06-06,0.00,0.00
4,IVL,0.1750,7200,1260.0,1134.00,2024-05-10,2025-05-27,2025-06-12,0.17,0.17
6,WHAIR,0.0582,60000,3492.0,3142.80,2024-05-13,2025-05-26,2025-06-25,0.00,0.00
2,DIF,0.2222,40000,8888.0,7999.20,2024-05-14,2025-05-14,2025-06-09,0.00,0.00
0,AIMIRT,0.2150,12500,2687.5,2418.75,2024-05-15,2025-05-19,2025-06-06,0.00,0.00
1,CPNREIT,0.0987,55000,5428.5,4885.65,2024-05-15,2025-05-19,2025-06-04,0.00,0.00


In [75]:
file_name = '20-div-25q1.xlsx'
xsl_file = os.path.join(xsl_path, file_name)

df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(xsl_file, index=False)

In [77]:
const.commit()
const.close()